# Práctica 3 - Redes Neuronales Residuales 

### Natalia Martínez García, Lucía Vega Navarrete
### Grupo: AP.11.06

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, metrics, optimizers, losses

### 1. Carga y preprocesado del dataset 

#### Carga 

In [4]:
DATA_PATH = "C:/Users/NataliaUDC/Downloads/higgs/HIGGS.csv.gz"

# Cargar el dataset 
higgs = pd.read_csv(DATA_PATH, compression="gzip", header=None)

print("Tamaño total del dataset (ejemplos, columnas):", higgs.shape)

# Separar características (X) y etiquetas (y)
#   - Columna 0: label
#   - Columnas 1-28: features
X = higgs.iloc[:, 1:].to_numpy(dtype=np.float32) 
y = higgs.iloc[:, 0].to_numpy(dtype=np.float32)

# División según especificaciones: train inicial, test final, extra intermedio
N_TRAIN = 2000000
N_TEST = 500000

X_train = X[:N_TRAIN]
y_train = y[:N_TRAIN]

X_test  = X[-N_TEST:]
y_test  = y[-N_TEST:]

X_extra = X[N_TRAIN:-N_TEST]
y_extra = y[N_TRAIN:-N_TEST]

print("División del dataset:")
print(f"Train: {X_train.shape}")
print(f"Test:  {X_test.shape}")
print(f"Extra: {X_extra.shape}")

Tamaño total del dataset (ejemplos, columnas): (11000000, 29)
División del dataset:
Train: (2000000, 28)
Test:  (500000, 28)
Extra: (8500000, 28)


#### Preprocesado

In [6]:
# Normalizar usando solo TRAIN
# Media y desviación por columna (feature) 
#    (x - media_train) / std_train

"""
Normalizamos solo en train porque cualquier información del test no puede influir en el entrenamiento (el modelo solo puede ver estadísticas del train).
Si usamos la media y desviación del test para normalizar, meteríamos información del futuro dentro del entrenamiento.

Eso se llama data leakage (filtración de información) y hace que tus resultados no sean realistas.
"""
mean_train = X_train.mean(axis=0) # axis = 0 para calcularlo por columnas (cada feature)
std_train  = X_train.std(axis=0)

# Evitar división por cero
std_train[std_train == 0] = 1.0

# Aplicar la transformación estándar: (x - media) / sd
X_train = (X_train - mean_train) / std_train
X_extra = (X_extra - mean_train) / std_train
X_test  = (X_test  - mean_train) / std_train

print("\n Normalización completada")
print(f"- Media global train (ya normalizado): {X_train.mean():.5f}")
print(f"- Std global train  (ya normalizado): {X_train.std():.5f}")

# Comprobación del desbalanceo de clases
def class_stats(name, y):
    n = len(y)
    n_signal = np.sum(y == 1)
    n_back   = np.sum(y == 0)
    print(f"\n{name}:")
    print(f"Signal = {n_signal} ({100*n_signal/n:.2f}%)")
    print(f"Background = {n_back}   ({100*n_back/n:.2f}%)")

print("\n Distribución de clases:")
class_stats("Train", y_train)
class_stats("Extra", y_extra)
class_stats("Test",  y_test)



 Normalización completada
- Media global train (ya normalizado): 0.00000
- Std global train  (ya normalizado): 1.00000

 Distribución de clases:

Train:
Signal = 1058818 (52.94%)
Background = 941182   (47.06%)

Extra:
Signal = 4505798 (53.01%)
Background = 3994202   (46.99%)

Test:
Signal = 264507 (52.90%)
Background = 235493   (47.10%)


### 2. Creación de red neuronal con capas residuales

In [7]:
input_dim = X_train.shape[1]  
print("Tamaño de entrada:", input_dim)

def residual_block(x, units, name):
    """
    Bloque residual con:
      - 2 capas lineales (Dense) internas
      - 1 conexión residual (skip connection)
    """
    x_skip = x  # conexión residual

    # 2 capas lineales antes de hacer la suma residual
    x = layers.Dense(units, activation="relu", name=f"{name}_dense1")(x)
    x = layers.Dense(units, activation="relu", name=f"{name}_dense2")(x)

    x = layers.Add(name=f"{name}_add")([x, x_skip]) # suma residual
    return x

inputs = keras.Input(shape=(input_dim,), name="higgs_input") 
x = layers.Dense(128, activation="relu", name="dense_in")(inputs) # Capa de entrada (proyección a dimensión oculta)
x = residual_block(x, units=128, name="res_block1") # Bloque residual 1 
x = residual_block(x, units=128, name="res_block2") # Bloque residual 2 
x = layers.Dense(64, activation="relu", name="dense_mid")(x) # (Opcional) capa intermedia antes de la salida
outputs = layers.Dense(1, activation="sigmoid", name="output")(x) # Capa de salida: 1 neurona con sigmoid para clasificación binaria

model = keras.Model(inputs=inputs, outputs=outputs, name="higgs_resnet") # Definir el modelo

model.summary() # Mostrar resumen para comprobar dimensiones

Tamaño de entrada: 28


Model: "higgs_resnet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ higgs_input         │ (None, 28)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_in (Dense)    │ (None, 128)       │      3,712 │ higgs_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res_block1_dense1   │ (None, 128)       │     16,512 │ dense_in[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res_block1_dense2   │ (None, 128)       │     16,512 │ res_block1_dense… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res_block1_add      │ (None, 128)       │          0 │ res_block1_dense… │
│ (Add)               │                   │            │ dense_in[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res_block2_dense1   │ (None, 128)       │     16,512 │ res_block1_add[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res_block2_dense2   │ (None, 128)       │     16,512 │ res_block2_dense… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res_block2_add      │ (None, 128)       │          0 │ res_block2_dense… │
│ (Add)               │                   │            │ res_block1_add[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_mid (Dense)   │ (None, 64)        │      8,256 │ res_block2_add[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         65 │ dense_mid[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 78,081 (305.00 KB)

 Trainable params: 78,081 (305.00 KB)

 Non-trainable params: 0 (0.00 B)

### 3. Entrenamiento de la red

En esta práctica hemos implementado el entrenamiento de la red siguiendo la estructura del tutorial “Escribir un ciclo de entrenamiento desde cero” de TensorFlow, proporcionado en el enunciado de esta práctica.
El código hace exactamente los mismos bloques lógicos propuestos en el tutorial, lo único diferente es que o ampliamos un poco para el cálculo de las métricas F1Score y balanced accurcay. 

1. Preparación de los datos

In [12]:
BATCH_SIZE = 4096

# Dataset de entrenamiento: barajado + dividido en batches
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=100_000, reshuffle_each_iteration=True).batch(BATCH_SIZE)

# Dataset de test: solo en batches, sin shuffle
val_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
val_dataset = val_dataset.batch(BATCH_SIZE)

print(train_dataset)
print(val_dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 28), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 28), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>


2. Definir loss, optimizer y métricas

In [ ]:
optimizer = optimizers.Adam(learning_rate=1e-3) # Optimizador
loss_fn = losses.BinaryCrossentropy() # Función de pérdida (binaria, porque es 0/1)


# Métricas para TRAIN
train_loss = metrics.Mean(name="train_loss")
train_accuracy = metrics.BinaryAccuracy(name="train_accuracy")

# Métricas para TEST
test_loss = metrics.Mean(name="test_loss")
test_accuracy = metrics.BinaryAccuracy(name="test_accuracy")
# Toca calcular el F1 a mano tb xq me estaba dando un error de dimensionalidad por usar la clase q lo hace directo de keras lmao 
test_precision = metrics.Precision(name="test_precision")
test_recall    = metrics.Recall(name="test_recall")

3. Esto está copiado tal cual el tutorial, se supone q hace el entrenamiento más rápido 

`@tf.function` convierte las funciones en grafos de TensorFlow optimizados, lo que hace que el entrenamiento sea más rápido y eficiente.

`train_step(x, y)` realiza un paso completo de entrenamiento (forward, cálculo de loss, gradientes, actualización de pesos y de métricas).

`test_step(x, y)` realiza la evaluación sobre un batch sin actualizar los pesos, solo actualizando las métricas de validación.

In [ ]:
@tf.function
def train_step(x, y):
    """Paso de entrenamiento: forward + loss + backprop + métricas train."""
    with tf.GradientTape() as tape:
        predictions = model(x, training=True)
        loss_value = loss_fn(y, predictions)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    # Actualizar métricas de entrenamiento
    train_loss.update_state(loss_value)
    train_accuracy.update_state(y, predictions)
    return loss_value
    
@tf.function
def test_step(x, y):
    """
    Paso de evaluación:
      - forward
      - actualizar métricas de test (loss, accuracy, precision, recall)
    Devuelve las predicciones para poder calcular balanced accuracy fuera.
    """
    predictions = model(x, training=False)
    loss_value = loss_fn(y, predictions)

    test_loss.update_state(loss_value)
    test_accuracy.update_state(y, predictions)
    test_precision.update_state(y, predictions)
    test_recall.update_state(y, predictions)

    return predictions

4. Bucle de epochs + F1 score + balanced accuracy + guardado 

La estructura es la misma que el tutorial:
* Loop de validación:
    * for x_batch_val, y_batch_val in test_ds: val_predictions = test_step(...)
* Leer métricas .result().
* Resetear métricas (reset_state()).
* Imprimir métricas + tiempo.

In [ ]:
import time

EPOCHS = 3 # ajusta según el tiempo que tengas

for epoch in range(EPOCHS):
    print("\nSTART OF EPOCH %d" % (epoch + 1,))
    print("")
    start_time = time.time()

    # FASE DE ENTRENAMIENTO
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        loss_value = train_step(x_batch_train, y_batch_train)

        # Log cada 200 batches 
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * BATCH_SIZE))

    # Mostrar las métricas al final de cada epoch 
    train_acc  = float(train_accuracy.result())
    train_loss_epoch = float(train_loss.result())
    print("\n--- Training metrics ---")
    print("Accuracy: %.4f" % train_acc)
    print("Loss: %.4f" % train_loss_epoch)

    # Reset training metrics at the end of each epoch
    train_loss.reset_state()
    train_accuracy.reset_state()

    # FASE DE EVALUACIÓN (TEST)
    # Esto lo usamos luego para el cálculo de balances accuracy 
    y_true_all = [] # todas las estiquetas reales 
    y_pred_all = [] # todas las predicciones binarias 

    for x_batch_val, y_batch_val in val_dataset:
        # test_step actualiza loss, accuracy, precision, recall
        val_predictions = test_step(x_batch_val, y_batch_val)

        # Guardar para balanced accuracy
        y_true_all.append(y_batch_val.numpy().astype(np.int32))
        y_pred_all.append((val_predictions.numpy() >= 0.5).astype(np.int32))

    y_true_all = np.concatenate(y_true_all, axis=0).reshape(-1)
    y_pred_all = np.concatenate(y_pred_all, axis=0).reshape(-1)

    # Métricas de test a partir de los objetos de Keras 
    val_loss = float(test_loss.result())
    val_acc = float(test_accuracy.result())
    precision = float(test_precision.result())
    recall = float(test_recall.result())
    f1_score = 2 * precision * recall / (precision + recall + 1e-8)

    # Reset de métricas de test para la siguiente época
    test_loss.reset_state()
    test_accuracy.reset_state()
    test_precision.reset_state()
    test_recall.reset_state()

    # Balanced Accuracy (a mano)
    # identificamos los índices donde las etiquetas son 0/1
    mask_0 = (y_true_all == 0) # pone a True todos los valores de la lista de etiquetas reales que valgan 0
    mask_1 = (y_true_all == 1) # pone a True todos los valores de la lista de etiquetas reales que valgan 1

    acc_class0 = np.mean(y_pred_all[mask_0] == y_true_all[mask_0])
    acc_class1 = np.mean(y_pred_all[mask_1] == y_true_all[mask_1])
    balanced_accuracy = (acc_class0 + acc_class1) / 2

    # Log de validación como en el tutorial, pero ampliado 
    print("\n--- Validation metrics ---")
    print("Loss: %.4f" % val_loss)
    print("Accuracy: %.4f" % val_acc)
    print("Precision: %.4f" % precision)
    print("Recall: %.4f" % recall)
    print("F1Score: %.4f" % f1_score)

    print("\n--- Balanced Accuracy ---")
    print("Class 0 (background): %.4f" % acc_class0)
    print("Class 1 (signal): %.4f" % acc_class1)
    print("Media: %.4f" % balanced_accuracy)

    print("")
    print("Time taken: %.2fs" % (time.time() - start_time))
    print("-"*50)

# Guardar el modelo entrenado
MODEL_PATH = "higgs_resnet_trained.keras"
model.save(MODEL_PATH)
print(f"\nModelo guardado en: {MODEL_PATH}")



START OF EPOCH 1

Training loss (for one batch) at step 0: 0.4690
Seen so far: 4096 samples
Training loss (for one batch) at step 200: 0.4703
Seen so far: 823296 samples
Training loss (for one batch) at step 400: 0.4748
Seen so far: 1642496 samples

--- Training metrics ---
Accuracy: 0.7720
Loss: 0.4665

--- Validation metrics ---
Loss: 0.4802
Accuracy: 0.7633
Precision: 0.7719
Recall: 0.7845
F1Score: 0.7781

--- Balanced Accuracy ---
Class 0 (background): 0.7395
Class 1 (signal): 0.7845
Media: 0.7620

Time taken: 8.02s
--------------------------------------------------

START OF EPOCH 2

Training loss (for one batch) at step 0: 0.4720
Seen so far: 4096 samples
Training loss (for one batch) at step 200: 0.4604
Seen so far: 823296 samples
Training loss (for one batch) at step 400: 0.4506
Seen so far: 1642496 samples

--- Training metrics ---
Accuracy: 0.7723
Loss: 0.4657

--- Validation metrics ---
Loss: 0.4805
Accuracy: 0.7641
Precision: 0.7700
Recall: 0.7899
F1Score: 0.7798

--- Bala